In [ ]:
## Tahap 5

import pandas as pd
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# --- Load queries JSON ---
json_path = '/content/drive/MyDrive/Penalaran Komputer/data/eval/queries_fidusia_baru.json'

with open(json_path) as f:
    queries = json.load(f)

# Buat mapping: query → ground_truth + true_amar
gt_map = {q['query']: [str(cid).lower() for cid in q['ground_truth']] for q in queries}
amar_map = {q['query']: q['true_amar'] for q in queries}

# --- Load prediksi CSV ---
df_pred = pd.read_csv('/content/drive/MyDrive/Penalaran Komputer/data/results/prediksi_amar_fidusia.csv')

# --- Evaluasi Retrieval ---
retrieval_metrics = []

for idx, row in df_pred.iterrows():
    query = row['query']

    raw_ids = str(row['top_5_case_ids'])

    # Deteksi pemisah list: bisa pakai koma atau titik koma
    if raw_ids.startswith('[') and raw_ids.endswith(']'):
        try:
            pred_ids = json.loads(raw_ids.replace("'", '"'))
        except:
            pred_ids = []
    else:
        pred_ids = [x.strip().lower() for x in raw_ids.split(';') if x.strip()]

    pred_ids = [x.lower() for x in pred_ids]
    true_ids = gt_map.get(query, [])

    true_positives = len(set(pred_ids) & set(true_ids))
    precision = true_positives / len(pred_ids) if pred_ids else 0
    recall = true_positives / len(true_ids) if true_ids else 0
    f1 = 2 * precision * recall / (precision + recall + 1e-6)

    retrieval_metrics.append({
        "query": query,
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    })

# Simpan hasil retrieval
retrieval_df = pd.DataFrame(retrieval_metrics)
retrieval_df.to_csv('/content/drive/MyDrive/Penalaran Komputer/data/eval/retrieval_metrics.csv', index=False)

# --- Evaluasi Klasifikasi Amar ---
df_pred['true_amar'] = df_pred['query'].map(amar_map)

# Drop baris yang tidak punya nilai untuk true/predicted
valid_rows = df_pred.dropna(subset=['true_amar', 'predicted_amar'])

# Pastikan semua string
y_true = valid_rows['true_amar'].astype(str)
y_pred = valid_rows['predicted_amar'].astype(str)


pred_metrics = {
    "accuracy": accuracy_score(y_true, y_pred),
    "precision": precision_score(y_true, y_pred, average='macro', zero_division=0),
    "recall": recall_score(y_true, y_pred, average='macro', zero_division=0),
    "f1_score": f1_score(y_true, y_pred, average='macro', zero_division=0)
}

# Simpan hasil klasifikasi
pd.DataFrame([pred_metrics]).to_csv('/content/drive/MyDrive/Penalaran Komputer/data/eval/prediction_metrics.csv', index=False)

# --- Output Ringkasan ---
print(f"\n✅ retrieval_metrics.csv saved.")
print(f"✅ prediction_metrics.csv saved.")
print(f"\n🎯 Accuracy: {pred_metrics['accuracy']:.2f} | Precision: {pred_metrics['precision']:.2f} | Recall: {pred_metrics['recall']:.2f} | F1: {pred_metrics['f1_score']:.2f}")
